Connect to Alpaca API paper trading account with key and secret key

In [1]:
from alpaca.data.historical import StockHistoricalDataClient

API_KEY = "PKPBPBL4QLI3Y5VONCGA"
SECRET_KEY = "Vrp8tsDgf7PIblEKoYRRaP0JQM6OavDQpFngbRqI"
 
historical_data = StockHistoricalDataClient(API_KEY, SECRET_KEY)

Request Stock data from Alpaca API and format to Pandas DataFrame for compatibility with Backtesting

In [124]:
# Alpaca Imports
from alpaca.data.requests import StockBarsRequest
from alpaca.data.timeframe import TimeFrame
import pandas as pd

# Plotly imports 
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Identify ticker symbol for Alpaca request
ticker_list = ["SPY"]

# Create Alpaca request for stock bar data
request_params = StockBarsRequest(
                        symbol_or_symbols=ticker_list,
                        timeframe=TimeFrame.Hour,
                        start=pd.Timestamp("2020-01-01", tz="America/New_York"),
                        end=pd.Timestamp("2021-01-01", tz="America/New_York"),
                 )

# Get stock bar data from Alpaca as dataframe object
bars = historical_data.get_stock_bars(request_params).df

# Reset bars index to only timestamp and drop symbol column
bars = bars.reset_index(level=0, drop=True)

# Rename the columns to match backtest columns
bars.rename(columns = {
       'open':'Open',
       'close':'Close',
       'high':'High',
       'low':'Low',
       'volume':'Volume',
       'symbol':'Symbol',
       'timestamp':'Date'
       }, inplace = True)

print(bars)

                               Open    High     Low   Close      Volume  \
timestamp                                                                 
2020-01-02 09:00:00+00:00  323.5200  323.86  323.52  323.83     28435.0   
2020-01-02 10:00:00+00:00  323.8200  323.98  323.82  323.91     15239.0   
2020-01-02 11:00:00+00:00  323.9400  323.94  323.63  323.63     17868.0   
2020-01-02 12:00:00+00:00  323.6100  323.85  323.61  323.85     64861.0   
2020-01-02 13:00:00+00:00  323.6899  324.08  322.83  324.03    366678.0   
...                             ...     ...     ...     ...         ...   
2020-12-31 20:00:00+00:00  372.7850  374.66  372.78  373.87  26078193.0   
2020-12-31 21:00:00+00:00  373.8500  374.69  373.84  374.30  23860544.0   
2020-12-31 22:00:00+00:00  374.2500  374.42  374.13  374.39   1660849.0   
2020-12-31 23:00:00+00:00  374.3900  374.41  374.39  374.40     18666.0   
2021-01-01 00:00:00+00:00  374.3900  374.41  374.26  374.39     39799.0   

                        

From the strategy classes for RSI, MACD and RSI/MACD

In [140]:
from backtesting import Strategy
from backtesting.lib import cross, crossover
import talib

class RSI_Strategy(Strategy):
    def init(self):
        self.rsi = self.I(talib.RSI, self.data.Close, 14)
    
    def next(self):       
        # If RSI crosses above 70, buy the asset
        if cross(self.rsi, 70):
            self.position.close()
            self.buy()

        # Else, if RSI crosses below 30, sell it
        if cross(self.rsi, 30):
            self.position.close()
            self.sell()
            
class MACD_Strategy(Strategy):
    def init(self):
        self.MACD = self.I(talib.MACD, self.data.Close, 12, 26, 9)
        print(self.MACD.shape)
    
    def next(self):       
        # If MACD line crosses above signal line, buy the asset
        if crossover(self.MACD[0], self.MACD[1]):
            self.position.close()
            self.buy()

        # Else, if MACD line crosses below signal line, sell the asset
        if crossover(self.MACD[1], self.MACD[0]):
            self.position.close()
            self.sell()

class RSI_MACD_Strategy(Strategy):
    def init(self):
        self.rsi = self.I(talib.RSI, self.data.Close, 14)
        self.MACD = self.I(talib.MACD, self.data.Close, 12, 26, 9)
    
    def next(self):       
        # If RSI crosses above 70 and MACD crosses from below, buy the asset
        if cross(self.rsi, 70) and (self.MACD[0], self.MACD[1]):
            self.position.close()
            self.buy()

        # Else, if RSI crosses below 30 and MACD crosses from above, sell it
        if cross(self.rsi, 30) and crossover(self.MACD[1], self.MACD[0]):
            self.position.close()
            self.sell()

Run backtest on strategy and plot results

In [141]:
from backtesting import Backtest

# Create a backtest object for all strategies
bt_RSI = Backtest(bars, RSI_Strategy, cash=10000, commission=.002)
bt_MACD = Backtest(bars, MACD_Strategy, cash=10000, commission=.002)
bt_RSI_MACD = Backtest(bars, RSI_MACD_Strategy, cash=10000, commission=.002)

# Run all backtests
bt_RSI.run()
bt_MACD.run()
bt_RSI_MACD.run()

# Plot all backtests
bt_RSI.plot()
bt_MACD.plot()
bt_RSI_MACD.plot()

# Print all backtest results
print(bt_RSI._results)
print(bt_MACD._results)
print(bt_RSI_MACD._results)

(3, 4053)
Start                     2020-01-02 09:00...
End                       2021-01-01 00:00...
Duration                    364 days 15:00:00
Exposure Time [%]                    99.03775
Equity Final [$]                  6456.544501
Equity Peak [$]                  11467.273986
Return [%]                         -35.434555
Buy & Hold Return [%]                15.61313
Return (Ann.) [%]                  -33.423468
Volatility (Ann.) [%]               17.666612
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -44.202823
Avg. Drawdown [%]                   -3.705896
Max. Drawdown Duration      294 days 03:00:00
Avg. Drawdown Duration       16 days 08:00:00
# Trades                                  305
Win Rate [%]                        30.163934
Best Trade [%]                       7.969484
Worst Trade [%]                     -6.205795
Avg. Trade [%]          

From these results we can see that although poor, the MACD/RSI strategy is the best performing wiith a toal of 159 trades and a win rate of 43%.  In order to try and improve the results, we can start by adding a trailing stop loss to the strategy as the size of the losses are quite significant.  In order to do this we will implement a TrailingStrategy from the Backtest library using the same MACD/RSI strategy as above.

In [194]:
from backtesting.lib import TrailingStrategy

class Trailing_RSI_MACD_Strategy(TrailingStrategy):

    stop_range = 0

    def init(self):
        super().init()
        self.rsi = self.I(talib.RSI, self.data.Close, 14)
        self.MACD = self.I(talib.MACD, self.data.Close, 12, 26, 9)
        super().set_trailing_sl(self.stop_range)
    
    def next(self):       
        super().next()
        # If RSI crosses above 70 and MACD crosses from below, buy the asset
        if cross(self.rsi, 70) and (self.MACD[0], self.MACD[1]):
            self.position.close()
            self.buy()

        # Else, if RSI crosses below 30 and MACD crosses from above, sell it
        if cross(self.rsi, 30) and crossover(self.MACD[1], self.MACD[0]):
            self.position.close()
            self.sell()

Lets backtest this strategy and see how minimizing losses helps to improve overall performance

In [195]:
# Create a backtest object for the trailing strategy
bt_Trailing_MACD_RSI = Backtest(bars, Trailing_RSI_MACD_Strategy, cash=10000, commission=.002)

# Run the backtest setting the stop loss range
stats = bt_Trailing_MACD_RSI.run(stop_range=10)

# Plot the backtest
bt_Trailing_MACD_RSI.plot()

# Print the backtest results
print(stats)

Start                     2020-01-02 09:00...
End                       2021-01-01 00:00...
Duration                    364 days 15:00:00
Exposure Time [%]                   78.657784
Equity Final [$]                  9460.775688
Equity Peak [$]                     11484.578
Return [%]                          -5.392243
Buy & Hold Return [%]                15.61313
Return (Ann.) [%]                   -5.023855
Volatility (Ann.) [%]               23.468211
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -24.219041
Avg. Drawdown [%]                   -3.974929
Max. Drawdown Duration      283 days 09:00:00
Avg. Drawdown Duration       19 days 16:00:00
# Trades                                  159
Win Rate [%]                        43.396226
Best Trade [%]                       5.759501
Worst Trade [%]                     -5.417547
Avg. Trade [%]                    